# CZ gate  

In [211]:
from system import *
#s1 =  system('2',MMA=True) 
s1 = load_object('o-x-o')

# $\Delta_{E\gamma} = \ \dfrac{\Delta_E}{\gamma}\, C^n \qquad \Delta_{e\gamma} = \ \dfrac{\Delta_e}{\gamma}\, C^m$

In [222]:
n = 1.2
m = 0

# Effective Hamiltonian Matrix obtain in $C \longrightarrow \infty$ limit

In [223]:
var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g',domain='positive')
var('DEt',domain='complex',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='complex',  latex_name =r'\tilde{\Delta }_e')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')

eff_hamiltonian_tayl = sg.copy( s1.eff_hamiltonian.parent().zero())
for diag in s1.pos_gs:
    print(f'Calculating {diag} out of {s1.pos_gs}...')
    #Take only one of the compl conj terms. Switch maybe second 0 to 1
    min_expr_len = 10**10
    for term in range(1):
        b = s1.eff_hamiltonian[diag,diag]

        b = b.operands()[1].operands()[term]
        #b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
        #b = b.subs(De=+0.5*I*gamma   +Det*gamma)
        b = b.subs(De=Deg*gamma*C^m)
        b = b.subs(DE=DEg*gamma*C^n)
        
        
        var('R_f',domain='positive')#ratio  (g_f/g)^2
        b = b.subs(g_f=g*sqrt(R_f))

        var('R_v',domain='positive',  latex_name =r'R_{\nu}') #ratio (v/g)^2
        b = b.subs(v=g*sqrt(R_v))

        var('r_b',domain='positive')
        b = b.subs(kappa_b=gamma/r_b)
        
        var('r_f',domain='positive',latex_name =r'r_f')
        b = b.subs(gamma_f=gamma*r_f)
        
        var('r_g',domain='positive',latex_name =r'r_g')
        b = b.subs(gamma_g=gamma*r_g)
        
        b = b.subs(g = sqrt(C*gamma*kappa_c)) 
        b = b.subs(r_b = c/(C*R_v))

        b = b*gamma  #Omega has to be substituted with Omega/gamma


        b = b._mathematica_().Factor()._sage_()
        b = MMA_simplify(b)#,full=True)
        #holds the smallest expression
        if len(str(b))<min_expr_len:
            min_expr_len = len(str(b))
            h_eff = b
    

    q = h_eff.subs(C=1/x)
    q = MMA_simplify(q)
    tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
    tayl = tayl.subs(x=1/C)
 
    
    #tayl =  (tayl + conjugate(tayl))/2      # try with + conjugate_transp

    #taylr = taylr.expand()
    tayl = MMA_simplify(tayl,full=True)
    eff_hamiltonian_tayl[diag,diag] = Omega^2/gamma * tayl

print('\nReal part of H_eff in ground state subspace taylor expanded in 1/C: ')
show(symround(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs],digits=4))

eff_hamiltonian_tayl = (eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs] + conjugate(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs]))/2

Calculating 0 out of [0, 1, 16, 17]...
Calculating 1 out of [0, 1, 16, 17]...
Calculating 16 out of [0, 1, 16, 17]...
Calculating 17 out of [0, 1, 16, 17]...

Real part of H_eff in ground state subspace taylor expanded in 1/C: 
symround: Deleted coefficient 4.441e-16
symround: Deleted coefficient 1.480e-16
symround: Deleted coefficient 9.252e-18
symround: Deleted coefficient 4.441e-16
symround: Deleted coefficient 1.480e-16
symround: Deleted coefficient 9.252e-18


[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          (c*(1.500*I*c + 0.5000*I) + 0.03125*I)*Omega^2/((1.500*I*DEg*c^2 - R_f*(c + 0.2500)^2/C^0.2000 + 0.5000*I*DEg*c + 0.03125*I*DEg + (c*(-0.7500*c - 0.2500) - 0.01562)*(r_f + r_g)/C^1.200)*C^1.200*gamma)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  0]
[                                                                                                                                                                                                                                                                                                                                                                                                                 

# CZ gate time

In [224]:
#l_0-l_1-l_2+l_3
denom = eff_hamiltonian_tayl[0,0]+eff_hamiltonian_tayl[3,3]-eff_hamiltonian_tayl[1,1]-eff_hamiltonian_tayl[2,2]

t_g = np.pi/denom
#t_g = MMA_simplify(t_g,full=False) #not full if too slow
t_g = SR(str(t_g).replace('Sqrt','sqrt')).simplify()

In [232]:
#show(symround((t_g)))
#GHZ REGIME 
rf =1 
de=122183415123789/100000000000000
r_v=1.01014311134038/c_k
#show(symround((t_g.subs(R_f =rf ,DEg=de,R_v=r_v))))
show(t_g.subs(c=np.pi,R_f=3,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2)))

6.283185307179586/(2*C^82.0*(1.6332963267948966*I*C + 33.403059578844484 + 6.144149445640851*I)*Omega^2/((7.3043232282282*I*C^84.2 - 0.75*C^84.0 + (149.38302374954353 + 27.477471648741183*I)*C^83.2 - (10.226171018936572 - 27.657914039364982*I)*C^83.0 + (99.72273498295571 - 261.9030123840012*I)*C^82.2 + (227.0284423059692 + 198.89923426949565*I)*C^82.0 + (184.2556050477521 + 70.15754688747364*I)*C^81.0)*gamma) + 2*C^82.0*(-1.6332963267948966*I*C + 33.403059578844484 - 6.144149445640851*I)*Omega^2/((-7.3043232282282*I*C^84.2 - 0.75*C^84.0 + (149.38302374954353 - 27.477471648741183*I)*C^83.2 - (10.226171018936572 + 27.657914039364982*I)*C^83.0 + (99.72273498295571 + 261.9030123840012*I)*C^82.2 + (227.0284423059692 - 198.89923426949565*I)*C^82.0 + (184.2556050477521 - 70.15754688747364*I)*C^81.0)*gamma) + 2*C^2*(24.576597782563404*I*C + 89.19472572963413 + 16.406452928428962*I)*Omega^2/((-54.954943297482366*I*C^4.2 - (199.44546996591146 + 36.68594401761759*I)*C^3.2 + (3*(3.391592653589793*C + 11.502900727884255 - 62.53625204635218*I)*C + 38.662236968837064*C + 25.809519142923044 - 140.31509377494717*I)*C^2)*gamma) - 2*C^2*(24.576597782563404*I*C - 89.19472572963413 + 16.406452928428962*I)*Omega^2/((54.954943297482366*I*C^4.2 - (199.44546996591146 - 36.68594401761759*I)*C^3.2 + (3*(3.391592653589793*C + 11.502900727884255 + 62.53625204635218*I)*C + 38.662236968837064*C + 25.809519142923044 + 140.31509377494717*I)*C^2)*gamma) - 16.406452928428934*I*Omega^2/(C^1.2*gamma*(-34.508702183652765/C^0.2000000000000002 - 25.809519142923044/C^1.2 - 36.68594401761758*I)) + 16.406452928428934*I*Omega^2/(C^1.2*gamma*(-34.508702183652765/C^0.2000000000000002 - 25.809519142923044/C^1.2 + 36.68594401761758*I)))

# Lindblau Operators in $C \longrightarrow \infty$ limit

Choose which of the Lindblau operators to view and which non zero element of it

In [226]:
lind_op_number = len(s1.lindblau_list)

eff_lind_taylor = []
eff_lind = []
eff_lind_coeff = []
for lind_op in range(lind_op_number ):#lind_op_number 
    print(f'{lind_op} out of {lind_op_number-1}')
    eff_lind_taylor.append([])
    eff_lind.append([])
    L_matrix = s1.eff_lindblau_list[lind_op]
    L_nonzeros = []
    L_nonzeros_pos = []
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))
    eff_lind_coeff.append(s1.L_coeffs[lind_op])
    for which in range(len(L_nonzeros)):

        L_elem = L_nonzeros[which]

        #L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
        #L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
        L_elem = L_elem.subs(De=Deg*gamma)
        L_elem = L_elem.subs(DE=DEg*gamma)

        #COUPLINGS

        L_elem = L_elem.subs(g_f=g*sqrt(R_f))

        L_elem = L_elem.subs(v=g*sqrt(R_v))

        #ABSORBTIONS
        L_elem = L_elem.subs(kappa_b=gamma/r_b)

        L_elem = L_elem.subs(gamma_g=gamma*r_g)

        L_elem = L_elem.subs(gamma_f=gamma*r_f)

        L_elem = L_elem.subs(g = sqrt(C*gamma*kappa_c)) 
        L_elem = L_elem.subs(r_b = c/(C*R_v))
        
        L_elem = L_elem._mathematica_().Factor()._sage_()


        L_elem = MMA_simplify(L_elem)#,full=True)

        L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))
        
        L_elem_show = symround(L_elem)
        
        eff_lind[lind_op].append(L_elem_show)
        
        var('x')
        q = L_elem.subs(C=1/x)
        q = MMA_simplify(q)
        tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
        tayl = tayl.subs(x=1/C)
        tayl = SR(str(tayl).replace('Sqrt','sqrt'))
        tayl_show = symround(tayl)
        
        eff_lind_taylor[-1].append(tayl_show)

0 out of 8
symround: Deleted coefficient -3.5e-18
symround: Deleted coefficient -8.9e-16
symround: Deleted coefficient 1.8e-15
symround: Deleted coefficient 1.3e-15
symround: Deleted coefficient -4.4e-16
symround: Deleted coefficient -5.6e-17
symround: Deleted coefficient -3.5e-18
symround: Deleted coefficient -8.9e-16
symround: Deleted coefficient 1.8e-15
symround: Deleted coefficient 1.3e-15
symround: Deleted coefficient -4.4e-16
symround: Deleted coefficient -5.6e-17
symround: Deleted coefficient 1.3e-16
symround: Deleted coefficient 4.4e-16
symround: Deleted coefficient 1.1e-16
symround: Deleted coefficient 8.7e-18
symround: Deleted coefficient -1.2e-16
symround: Deleted coefficient 6.9e-17
symround: Deleted coefficient -6.9e-33
symround: Deleted coefficient 3.5e-33
symround: Deleted coefficient -8.7e-34
symround: Deleted coefficient 4.3e-34
symround: Deleted coefficient -1.1e-16
symround: Deleted coefficient -4.3e-19
symround: Deleted coefficient 2.8e-32
symround: Deleted coeffici

# See results

In [227]:
#show((eff_lind_taylor[2][3]))

In [228]:
#show(eff_lind[0][3])

In [229]:
lindop = 0
sub_bool = True
a = eff_lind_taylor[lindop][0]
err = t_g*a*conjugate(a)
err = SR(str(MMA_simplify( err,full=False )).replace('Sqrt','sqrt')).simplify()
show( symround( err ) )
print(f'coeff:')
show(eff_lind_coeff[lindop])

Interrupting Mathematica...


KeyboardInterrupt: Ctrl-c pressed while running Mathematica

In [230]:
lindop =0
sub_bool = True
a = eff_lind_taylor[lindop][0]
err = t_g*a*conjugate(a)
if sub_bool ==True:
    err = err.subs(c=np.pi,R_f=3,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2))
    err = symround(err)
err = SR(str(MMA_simplify( err,full=False )).replace('Sqrt','sqrt')).simplify()
show( symround( err ) )
print(f'coeff:')
show(eff_lind_coeff[lindop])

2176/((C^2*(4.8*I*C + 18. + 3.2*I)/(-5.4*I*C^4.2 + C^4 - (20. + 3.6*I)*C^3.2 + (7.1 - 18.*I)*C^3 + (2.5 - 14.*I)*C^2) + C^2*(-4.8*I*C + 18. - 3.2*I)/(5.4*I*C^4.2 + C^4 - (20. - 3.6*I)*C^3.2 + (7.1 + 18.*I)*C^3 + (2.5 + 14.*I)*C^2) + C*((0.45 + 0.045*I)*C + 0.73 + 9.2*I)/(C^3 + (5.5 + 20.*I)*C^2 - (13. - 44.*I)*C + 6.9 + 26.*I) + C*((0.45 - 0.045*I)*C + 0.73 - 9.2*I)/(C^3 + (5.5 - 20.*I)*C^2 - (13. + 44.*I)*C + 6.9 - 26.*I) + 16.*I/(C^1.2*(34./C^0.20 + 26./C^1.2 + 36.*I)) + 16.*I/(C^1.2*(-34./C^0.20 - 26./C^1.2 + 36.*I)))*C^1.4*(34./C^0.19 + 26./C^1.2 + 36.*I)*(34./C^0.19 + 26./C^1.2 - 36.*I))

coeff:


sqrt(kappa_c)